In [66]:
import pandas as pd
import requests
import json
from tqdm import tqdm
from sqlalchemy import create_engine
import boto3

from keys import *

In [92]:
types = {'Name':'str','Code':'str'}
name_list = list(pd.read_csv("~/airflow/stock_data/data/code.csv")['Name'])
name_df = pd.read_csv("~/airflow/stock_data/data/code.csv", dtype=types).rename(columns={"Name":"name","Code":"stock_code"})

In [68]:
url = "https://openapi.naver.com/v1/search/news.json"

header = {
    "X-Naver-Client-Id" : "azhP2a68ejoD_N1Bwp55",
    "X-Naver-Client-Secret":"I9LYuloz92"
}

In [69]:
# rds와 연결
user = 'ants'
password = rds_password()
host= end_point()
port = 3306
database = 'datawarehouse'
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

In [74]:
# for x in tqdm(name_list):    
param = {
    "query" : "삼성전자",
    "display" : 100,
    "start" : 1,
    "sort" : "sim",
}
r = requests.get(url, params=param, headers=header)

In [94]:
data = r.json()['items']
# with open(f"./news_data/{x}.json", "w", encoding='utf-8') as f:
#     json.dump(data, f, ensure_ascii=False, indent=4)
df = pd.DataFrame(data)
df['pubDate'] = pd.to_datetime(df['pubDate'], format='%a, %d %b %Y %H:%M:%S %z')
rds_df = df[df['pubDate'] >= "20240830"]
rds_df['name'] = "삼성전자"
rds_df.drop(columns="link", inplace=True)

/tmp/ipykernel_2351341/3388790679.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rds_df['name'] = "삼성전자"
/tmp/ipykernel_2351341/3388790679.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rds_df.drop(columns="link", inplace=True)


In [97]:
rds_df = pd.merge(rds_df, name_df, on = 'name', how='left')
columns = ['stock_code','name','pubDate','title','description','originallink']
rds_df = rds_df[columns]

In [98]:
rds_df

,stock_code,name,pubDate,title,description,originallink
0,005930,삼성전자,2024-08-30 10:03:00+09:00,"&quot;<b>삼성전자</b>, 노키아 모바일 네트워크 인수에 관심&quot; &...",<b>삼성전자</b>가 핀란드의 통신장비업체 노키아의 모바일 네트워크 자산 인수에 ...,https://news.tvchosun.com/site/data/html_dir/2...
1,005930,삼성전자,2024-08-30 09:29:00+09:00,"<b>삼성전자</b> AI 가전, '스마트 포워드'로 다양한 신기능 추가",<b>삼성전자</b>는 자사 소프트웨어 정기 업데이트 서비스인 '스마트 포워드(Sm...,https://zdnet.co.kr/view/?no=20240830091122
2,005930,삼성전자,2024-08-30 08:14:00+09:00,"<b>삼성전자</b>, '생성형 배경 화면' '컵 세척 코스' 등 AI 신기능 추가",<b>삼성전자</b>는 소프트웨어 정기 업데이트 서비스인 '스마트 포워드(Smart...,https://www.dailian.co.kr/news/view/1401567/?s...
3,005930,삼성전자,2024-08-30 13:55:00+09:00,"블룸버그 “<b>삼성전자</b>, 노키아 통신장비 인수에 관심”",<b>삼성전자</b>가 글로벌 3위 통신장비업체 노키아의 통신장비 자산 인수에 관심...,https://www.donga.com/news/Economy/article/all...
4,005930,삼성전자,2024-08-30 10:46:00+09:00,"&quot;<b>삼성전자</b>, 노키아 모바일 네트워크 인수에 관심&quot;",<b>삼성전자</b>가 핀란드 통신장비업체 노키아의 모바일 네트워크 자산 인수에 관...,https://www.ytn.co.kr/_ln/0104_202408301046116168
...,...,...,...,...,...,...
75,005930,삼성전자,2024-08-30 09:20:00+09:00,"<b>삼성전자</b>, 신기능 업데이트로 AI 가전 사용성↑",<b>삼성전자</b>가 소프트웨어 정기 업데이트 서비스인 '스마트 포워드'를 통해 ...,http://www.dizzotv.com/site/data/html_dir/2024...
76,005930,삼성전자,2024-08-30 09:06:00+09:00,"<b>삼성전자</b>, AI 주방가전 '스마트포뭐드' 업데이트로 새기능 단다",<b>삼성전자</b>가 소프트웨어 정기 업데이트 서비스인 '스마트 포워드'(Smar...,http://www.seoulwire.com/news/articleView.html...
77,005930,삼성전자,2024-08-30 09:42:00+09:00,"<b>삼성전자</b>, 냉장고·식세기 등에 AI 가전 신기능 업데이트",<b>삼성전자</b> 모델이 '비스포크 AI 패밀리허브' 냉장고의 '생성형 배경 화...,http://www.smedaily.co.kr/news/articleView.htm...
78,005930,삼성전자,2024-08-30 15:13:00+09:00,'패럴림픽 후원사' <b>삼성전자</b>가 전세계 청소년들 파리로 모은 이유,올림픽·패럴림픽 공식 후원사인 <b>삼성전자</b>가 프랑스 파리에서 CSR(사회공...,http://news.mt.co.kr/mtview.php?no=20240830150...


In [80]:
rds_df.to_sql('news', index=False, if_exists="append", con=engine)

80

In [77]:
rds_df.columns

Index(['title', 'originallink', 'description', 'pubDate', 'name',
       'stock_code'],
      dtype='object')

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype                    
---  ------        --------------  -----                    
 0   title         100 non-null    object                   
 1   originallink  100 non-null    object                   
 2   link          100 non-null    object                   
 3   description   100 non-null    object                   
 4   pubDate       100 non-null    datetime64[ns, UTC+09:00]
dtypes: datetime64[ns, UTC+09:00](1), object(4)
memory usage: 4.0+ KB
